In [ ]:
import numpy
from numpy.polynomial.polynomial import polyvalfromroots

from matplotlib import pyplot as plt
%matplotlib inline

from tbcontrol.plotting import cross_axis

from ipywidgets import interact



# Respuesta en frecuencia

La respuesta en frecuencia de un sistema hace referencia a cómo reacciona en estado estacionario ante entradas sinoidales. Analizar dicha respuesta para sistemas en lazo abierto permite determinar estabilidad de los mismos en lazo cerrado.


In [ ]:
# Polos Ceros y Ganancia de una FdT
zeros = [1]
poles = [-1 + 1j, -1 - 1j]
gain = 1
def G(s):# función G para evaluar en frecuencias
    return gain*polyvalfromroots(s, zeros)/polyvalfromroots(s, poles)

In [ ]:
#funciones para graficar
def plotcomplex(curve, color='blue', marker=None):
    plt.plot(numpy.real(curve), numpy.imag(curve), color=color, marker=marker)
def plotpz():
    for p in poles:
        plotcomplex(p, color='red', marker='x')
    for z in zeros:
        plotcomplex(z, color='red', marker='o')
        
def plotsituation(contour):
    plotcomplex(contour)
    plotcomplex(G(contour), color='red')
    plotpz()
    cross_axis()

In [ ]:
theta = numpy.linspace(0, 2*numpy.pi, 1000)

In [ ]:
def argumentprinciple(centerreal=(-2., 2.), centerimag=(-2., 2.), radius=(0.5, 3)):
    contour = radius*numpy.exp(1j*theta) + centerreal + 1j*centerimag
    plotsituation(contour)    

In [ ]:
interact(argumentprinciple)